# Track a folder of files

```{warning}

Currently only supports data folders that are located in the configured storage.
```

In [ ]:
!lamin load mydata

In [ ]:
import lamindb as ln

ln.track()

## Track a folder and its containing files

In [ ]:
ln.dev.datasets.generate_cell_ranger_files(
    "sample_001", ln.setup.settings.instance.storage.root
)

In [ ]:
!ls -l './mydata/sample_001/'

Let's pass a directory path to `ln.Folder`, which creates a Folder record:

In [ ]:
folder = ln.Folder(folder="./mydata/sample_001/")

folder

Meanwhile creates file records correspond to each of the file inside the Folder:

In [ ]:
folder.files

In [ ]:
ln.add(folder)

## What happens under the hood?

### In the SQL database

1. A `Folder` entry
2. 15 `File` entries correspond to 15 files inside the directory
2. A `Notebook` entry
3. A `Run` entry

All three entries are linked so that you can find the file using any of the metadata fields.

In [ ]:
ln.select(ln.Folder, name=folder.name).one()

In [ ]:
ln.select(ln.Folder).join(ln.File.folders).where(ln.Folder.name == "sample_001").df()

In [ ]:
ln.select(ln.schema.Notebook, id=ln.context.transform.id).one()

In [ ]:
ln.select(ln.schema.Run, id=ln.context.run.id).one()

## View the directory tree

In [ ]:
folder.tree()

## Find and retrieve files in folder

### Retrieve files from a folder

In [ ]:
with ln.Session() as ss:
    folder = ss.select(ln.Folder, name="sample_001").first()
    files = folder.files

In [ ]:
files[:2]

### Retrieve files via it's relative path to the directory

In [ ]:
folder.get(relpath="raw_feature_bc_matrix/features.tsv.gz")

In [ ]:
folder.get(relpath=["analysis/analysis.csv", "raw_feature_bc_matrix/features.tsv.gz"])

In [ ]:
folder.get(relpath="raw_feature_bc_matrix")

In [ ]:
folder.get(relpath="raw_feature_bc_matrix", suffix=".mtx.gz")

Query a specific file from a folder using `ln.select`:

In [ ]:
ln.select(ln.File, name="metrics_summary").join(ln.File.folders).where(
    ln.Folder.name == "sample_001"
).df()